<img src="../exploratoryAnalysis/zona_jobs.png">

# Trabajo Practico N° 2

### Aqui procederemos a la limpieza de datos

#### 1-1 Carga de datos

In [1]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.mlab import bivariate_normal
%matplotlib inline
import pandas as pd
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
import seaborn as sns
warnings.filterwarnings('ignore')
import plotly

In [2]:
# Carga de archivos
post_educacion  = pd.read_csv('../data/fiuba_1_postulantes_educacion.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})

In [17]:
post_gen_edad   = pd.read_csv('../data/fiuba_2_postulantes_genero_y_edad.csv', dtype = {
    'sexo': 'category'    
})

In [18]:
vistas          = pd.read_csv('../data/fiuba_3_vistas.csv'       , parse_dates=['timestamp'])

In [19]:
postulaciones   = pd.read_csv('../data/fiuba_4_postulaciones.csv', parse_dates=['fechapostulacion'])

In [20]:
avisos_online   = pd.read_csv('../data/fiuba_5_avisos_online.csv')

In [21]:
avisos_detalles = pd.read_csv('../data/fiuba_6_avisos_detalle.csv', dtype= {
    'nombre_zona': 'category',
    'ciudad': 'category',
    'tipo_de_trabajo': 'category',
    'nivel_laboral': 'category',
})

In [3]:
# Carga de archivos
post_educacion_pre  = pd.read_csv('../data/pre_abril/fiuba_1_postulantes_educacion.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})

In [22]:
post_gen_edad_pre   = pd.read_csv('../data/pre_abril/fiuba_2_postulantes_genero_y_edad.csv', dtype = {
    'sexo': 'category'    
})

In [ ]:
vistas_pre          = pd.read_csv('../data/pre_abril/fiuba_3_vistas.csv'       , parse_dates=['timestamp'])

In [ ]:
postulaciones_pre   = pd.read_csv('../data/pre_abril/fiuba_4_postulaciones.csv', parse_dates=['fechapostulacion'])

In [ ]:
avisos_online_pre   = pd.read_csv('../data/pre_abril/fiuba_5_avisos_online.csv')

In [ ]:
avisos_detalles_pre = pd.read_csv('../data/pre_abril/fiuba_6_avisos_detalle.csv', dtype= {
    'nombre_zona': 'category',
    'ciudad': 'category',
    'tipo_de_trabajo': 'category',
    'nivel_laboral': 'category',
})

In [4]:
# Carga de archivos
post_educacion_post  = pd.read_csv('../data/post_abril/fiuba_1_postulantes_educacion.csv', dtype = {
    'nombre': 'category',
    'estado': 'category'
})

In [13]:
vistas_post          = pd.read_csv('../data/post_abril/fiuba_3_vistas.csv'       , parse_dates=['timestamp'])

In [ ]:
avisos_detalles_post = pd.read_csv('../data/post_abril/fiuba_6_avisos_detalle.csv', dtype= {
    'nombre_zona': 'category',
    'ciudad': 'category',
    'tipo_de_trabajo': 'category',
    'nivel_laboral': 'category',
})

In [10]:
frames = [ post_educacion, post_educacion_pre, post_educacion_post ]
post_ed_total = pd.concat(frames)
post_ed_total.head()

,idpostulante,nombre,estado
0,NdJl,Posgrado,En Curso
1,8BkL,Universitario,En Curso
2,1d2B,Universitario,En Curso
3,NPBx,Universitario,En Curso
4,NPBx,Master,En Curso


In [11]:
frames = [ post_gen_edad, post_gen_edad_pre]
post_gen_edad_total = pd.concat(frames)
post_gen_edad_total.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [12]:
frames = [ vistas_pre, vistas_post]
vistas_total = pd.concat(frames)
vistas_total.head()

,idAviso,timestamp,idpostulante
0,1112400775,2018-04-06 05:43:47.445,kPjr2K0
1,1112400775,2018-04-06 05:43:48.330,kPjr2K0
2,1112426664,2018-04-06 05:43:50.042,GNOwZmb
3,1112426664,2018-04-06 05:43:51.171,GNOwZmb
4,1112445738,2018-04-06 05:43:46.371,lDLxajz


In [14]:
frames = [ postulaciones, postulaciones_pre]
postulaciones_total = pd.concat(frames)
postulaciones_total.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [15]:
frames = [ avisos_online, avisos_online_pre]
avisos_online_total = pd.concat(frames)
avisos_online_total.head()

,idaviso
0,1112355872
1,1112335374
2,1112374842
3,1111984070
4,1111822480


In [16]:
frames = [ avisos_detalles, avisos_detalles_pre, avisos_detalles_post]
avisos_detalles_total = pd.concat(frames)
avisos_detalles_total.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [ ]:
# Recorto la columna idpais ya que no es necesaria.
avisos_detalles_total.drop('idpais', axis=1, inplace=True)

In [ ]:
# Agrupo las áreas parecidas.
areas_agrupadas = pd.read_csv('../extraData/areas_agrupadas.csv')
avisos_detalles_total = pd.merge(avisos_detalles, areas_agrupadas, how='inner', on='nombre_area')
avisos_detalles_total['nombre_area'] = avisos_detalles['nuevo_nombre_area']
avisos_detalles_total.drop('nuevo_nombre_area', axis=1, inplace=True)

In [ ]:
# Convierto el nombre del area a category.
avisos_detalles_total['nombre_area'] = avisos_detalles['nombre_area'].astype('category')

In [ ]:
# Agrego tipo categorico y modifico el tipo de dato de la fecha.
post_gen_edad_total.fechanacimiento = post_gen_edad.fechanacimiento.apply(pd.to_datetime, errors='coerce')

now = pd.Timestamp(dt.datetime.now())
pd.to_datetime(now)
post_gen_edad_total['edad'] = (now - pd.to_datetime(post_gen_edad['fechanacimiento'])).astype('<m8[Y]')

In [ ]:
# Renombramos algunas columnas para unificar criterios.
vistas_total.rename(columns={"idAviso":"idaviso"}     , inplace=True)
vistas_total.rename(columns={"timestamp":"fechavista"}, inplace=True)

# Cambiar el tipo de dato de la fecha de vista.
vistas_total[['fechavista']] = vistas[['fechavista']].apply(pd.to_datetime)

## Luego de mergear los df debo quitar cols

### Para poder extraer info innecesaria y poder utilizar en el entrenamiento